### <p style="text-align: center; margin: 0;"> Міністерство освіти і науки України </p>
### <p style="text-align: center; margin: 0;"> Національний технічний університет України </p>
### <p style="text-align: center; margin: 0;"> «Київський політехнічний інститут імені Ігоря Сікорського» </p>
### <p style="text-align: center; margin: 0;"> «Інститут прикладного системного аналізу» </p>
### <p style="text-align: center; margin: 0;"> Кафедра математичних методів системного аналізу </p>
<br>

## <p style="text-align: center; margin: 0;"> Звіт </p>
### <p style="text-align: center; margin: 0;"> про виконання лабораторної роботи №2 </p>
### <p style="text-align: center; margin: 0;"> з дисципліни </p>
### <p style="text-align: center; margin: 0;"> «Розпізнавання образів» </p>
<br>

### <p style="text-align: right; margin: 0;"> Виконали: студенти ІV курсу &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> групи  КА-76  &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> Панасюк Я.І. &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> та групи  КА-76  &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> Іванов С. &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> Перевірила: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;</p>
### <p style="text-align: right; margin: 0;"> Дідковська М.В. &emsp;&emsp;</p>
<br>

### <p style="text-align: center; margin: 0;"> Київ – 2020 </p>
<br>
<br>

<br>

# Завдання

Кожен учасник або учасниця команди спершу обирає дескриптор (один із розглянутих у лекціях або ж знайдений окремо) та предмет на прикладі якого відбуватиметься дослідження. Враховуючи що вас багато, будь ласка обирайте унікальніші предмети за улюблену чашку/телефон/мишку. Маючи те і інше напоготові кожен учасник бригади має зняти не менше сотні фото предмета, варіюючи його розміщення та ракурс в кадрі, освітлення, наявність візуальних перешкод, зашакаленість зображення, фокусну віддаль та тремтіння рук. Сотня фото обраного предмету на однаковій сцені з однаковою якістю зйомки, але з різних ракурсів на жаль не підійде, постарайтесь наполегливо варіювати сцени і умови зйомки. До цих фото варто додати невелику підбірку зображень, що не містять предмет, або ж містять предмет візуально подібний до вашого, штук 20 повинно вистачити, якщо залишиться натхнення можна й більше. Після чого ми нарешті дійшли до цікавого, а саме до дослідження:
Вам потрібно згенерувати обраний дескриптор для обраного предмета, після чого з його допомогою розпізнати об’єкт на всій тестовій вибірці збираючи при цьому такі метрики: відносна кількість правильно суміщених ознак, похибка локалізації (відстань між реальним розміщенням предмета в кадрі та розпізнаним) та відносний час обробки фото в залежності від розміру зображення. Метрики мають зберегтись у файлику для подальших досліджень. 
Наступним кроком ви обмінюєтесь об’єктом з колегою, і уже маючи готову збиралку метрик, обчислюєте їх для предмета вашого сусіда, таким чином у вас збирається 9 наборів даних, по три на дескриптор. 
Самою ж ідеєю лаби є дослідити розбіжності у роботі ваших дескрипторів та виконати порівняльний аналіз їх поведінки, сформулювати висновки з викладками і прикладами так аби було зрозуміло вам та, сподіваюсь, усім вашим колегам. Таким чином кінцевим результатом буде від вас гуглдок з описом виняткових особливостей, сильних та слабких сторін дескриптора і обгрунтуванням чому вони поводяться саме так.


<br>

# Хід роботи

### Вибірки об'єктів

In [1]:
import glob
import cv2 

duck_train_pth = glob.glob("duck_ds/train/*.jpg")
duck_test_pth = glob.glob("duck_ds/test/*.jpg")
duck_ds_train = [cv2.imread(_) for _ in duck_train_pth]
duck_ds_test = [cv2.imread(_) for _ in duck_test_pth]

<br>

### Дескриптори

ORB (Oriented FAST and Rotated BRIEF) 

In [2]:
import numpy as np 
import cv2 
    
def ORB_descript(query_img, train_img, show_final_img=False):    
    # Read the query image as query_img 
    # and traing image This query image 
    # is what you need to find in train image 
   
    # Convert it to grayscale 
    query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY) 
    train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY) 
   
    # Initialize the ORB detector algorithm 
    orb = cv2.ORB_create() 
   
    # Now detect the keypoints and compute 
    # the descriptors for the query image 
    # and train image 
    queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw,None) 
    trainKeypoints, trainDescriptors = orb.detectAndCompute(train_img_bw,None) 
  
    if(queryDescriptors is None or trainDescriptors is None):
        return 0, 966
    
    # Initialize the Matcher for matching 
    # the keypoints and then match the 
    # keypoints 
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True) 
    matches = matcher.match(queryDescriptors,trainDescriptors) 

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    
    if(show_final_img):
        # draw the matches to the final image 
        # containing both the images the drawMatches() 
        # function takes both images and keypoints 
        # and outputs the matched query image with 
        # its train image 
        final_img = cv2.drawMatches(query_img, queryKeypoints,  
        train_img, trainKeypoints, matches[:20],None) 
   
        final_img = cv2.resize(final_img, (1000,650)) 
  
        # Show the final image 
        cv2.imshow("Matches", final_img) 
        cv2.waitKey(3000)
    
    # Metrics
    # the relative number of correctly matched features
    rel_numb = len(matches)*2 / (len(queryDescriptors) + len(trainDescriptors))    
    
    # localization inaccuracy
    distances = [_.distance for _ in matches]
    avr_dist = sum(distances)/len(distances)
    
    return rel_numb, avr_dist

### Метрики

In [3]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from pathlib import Path

def get_metrics(descriptor, ds_train, ds_test, ds_name, plotting_graphs=True):
    """It creates files with "the relative number of correctly matched features" and "localization inaccuracy" metrics"""
    
    rel_numb_of_correct_mathced_feat=[]
    distances=[]
    
    for train_img in ds_train:
        distances_to_train_img =[]
        rel_numb_for_train_img =[]
        for test_img in ds_test:
            rel_numb, avr_dist = descriptor(train_img, test_img)
        
            rel_numb_for_train_img.append(rel_numb)
            distances_to_train_img.append(avr_dist)
    
        rel_numb_of_correct_mathced_feat.append(rel_numb_for_train_img)    
        distances.append(distances_to_train_img)

    # Create dirs for output data
    Path(f"{ds_name}/metrics/data").mkdir(parents=True, exist_ok=True)
    Path(f"{ds_name}/metrics/graphs").mkdir(parents=True, exist_ok=True)
    
    # write metrics for every train img
    for i in range(len(ds_train)):
        with open(f"{ds_name}/metrics/data/train_img{i}_rel_numb.txt", "w") as file1: 
            file1.write("\n".join([str(_) for _ in rel_numb_of_correct_mathced_feat[i]]))
        with open(f"{ds_name}/metrics/data/train_img{i}_avr_dist.txt", "w") as file2:  
            file2.write("\n".join([str(_) for _ in distances[i]]))
    
    
    if(plotting_graphs):
        arr=[_ for _ in range(len(ds_test))]
        
        for i in range(len(ds_train)):
            output_file(f"{ds_name}/metrics/graphs/train_img{i}_rel_numb_graph.html")
            p = figure(title=f"the relative number of correctly matched features from train_img_{i}" , x_axis_label='test_img_id', y_axis_label='rel numb')
            p.line(arr, rel_numb_of_correct_mathced_feat[i], legend_label="Temp.", line_width=2)
            show(p)
        
        for i in range(len(ds_train)):
            output_file(f"{ds_name}/metrics/graphs/train_img{i}_distance_graph.html")
            p = figure(title=f"Distance to train_img_{i}" , x_axis_label='test_img_id', y_axis_label='distance')
            p.line(arr, distances[i], legend_label="Temp.", line_width=2)
            show(p)  

### Отримання метрик

In [4]:
get_metrics(ORB_descript, duck_ds_train, duck_ds_test, "duck_ds")

### Результати

Отримані метрики можно побачити у файлах \[Назва датасету\]\_\[Метрика\]  
Граффіки на основі отриманих даних \[Назва датасету\]\_\[Метрика\]\_\[Номер тренованого зображення в масиві\]

Щоб вивести дескпиптор конкретного зображення, введіть номер тренованого й тестового зображення у масиві

In [5]:
rel_numb, avr_dist  = ORB_descript(duck_ds_train[1], duck_ds_test[40], show_final_img=True)

### Аналіз